In [1]:
import sqlite3
import time
import statistics

In [2]:
db_path = 'Dirigo.sqlite'

query = '''
WITH IdentifiedTruck AS (
  SELECT Object_id AS TruckID
  FROM OCEL_Truck
  WHERE LPT = '926VFT'
),
LastDropTime AS (
  SELECT MIN(Timestamp) AS DroppedAt
  FROM OCEL_O2O
  WHERE Source_object_id = (SELECT TruckID FROM IdentifiedTruck)
    AND O2O_qualifier = 'Dropped truck from pickup plan'
),
NextAssignedPickup AS (
  SELECT Target_object_id AS AssignedPickupPlan, Timestamp
  FROM OCEL_O2O
  WHERE Source_object_id = (SELECT TruckID FROM IdentifiedTruck)
    AND O2O_qualifier = 'Assigned truck for pickup plan'
    AND Timestamp > (SELECT DroppedAt FROM LastDropTime)
  ORDER BY Timestamp ASC
  LIMIT 1
)
SELECT AssignedPickupPlan
FROM NextAssignedPickup;
'''
repeat = 1000

conn = sqlite3.connect(db_path)
cursor = conn.cursor()


durations = []

for i in range(repeat):
    start_time = time.time()
    cursor.execute(query)
    cursor.fetchall()  
    end_time = time.time()
    
    duration = (end_time - start_time) * 1000  
    durations.append(duration)
    # print(f"Run {i+1}: {duration:.4f} ms")


avg = statistics.mean(durations)
std_dev = statistics.stdev(durations)

print("\n=== Execution Statistics (Dirigo) ===")
print(f"Average execution time: {avg:.4f} ms")
print(f"Standard deviation: {std_dev:.4f} ms")


cursor.close()
conn.close()


=== Execution Statistics (Dirigo) ===
Average execution time: 0.0662 ms
Standard deviation: 0.0301 ms


In [3]:
db_path = 'ACEL.sqlite'

query = """
WITH IdentifiedTruck AS (
  SELECT ObjectId AS TruckID
  FROM ACEL_Objects
  WHERE LPT = '926VFT'
),
TruckEvents AS (
SELECT e.*
FROM ACEL_Events e
WHERE EXISTS (
  SELECT 1
  FROM json_each(REPLACE(e.Objects, '''', '"'))
  WHERE json_each.value = (SELECT TruckID FROM IdentifiedTruck))
 ),
TruckFirstAssignTime AS (
  SELECT MIN(Timestamp) AS FirstTime
  FROM TruckEvents
  WHERE Activity = 'Assign Truck'
  ORDER BY Timestamp
),
SecondAssignEvent AS (
  SELECT *
  FROM TruckEvents
  WHERE Activity = 'Assign Truck'
  AND Timestamp > (SELECT FirstTime FROM TruckFirstAssignTime)
  ORDER BY Timestamp
  LIMIT 1
)
SELECT json_extract(jc.value, '$.NewValue') AS NewStatus
FROM SecondAssignEvent, json_each(REPLACE(ObjectChanges, '''', '"')) AS jc
WHERE json_extract(jc.value, '$.Attribute') = 'Pickup Plan ID';
"""

repeat = 1000

conn = sqlite3.connect(db_path)
cursor = conn.cursor()


durations = []

for i in range(repeat):
    start_time = time.time()
    cursor.execute(query)
    cursor.fetchall()  
    end_time = time.time()
    
    duration = (end_time - start_time) * 1000  
    durations.append(duration)
    # print(f"Run {i+1}: {duration:.4f} ms")


avg = statistics.mean(durations)
std_dev = statistics.stdev(durations)

print("\n=== Execution Statistics (ACEL) ===")
print(f"Average execution time: {avg:.4f} ms")
print(f"Standard deviation: {std_dev:.4f} ms")


cursor.close()
conn.close()


=== Execution Statistics (ACEL) ===
Average execution time: 0.2732 ms
Standard deviation: 0.0272 ms


In [4]:
db_path = 'DOCEL.sqlite'

query = """
WITH IdentifiedTruck AS (
  SELECT ObjectID AS TruckID
  FROM DOCEL_Truck_Static
  WHERE LPT = '926VFT'
),
TruckPickupPlans AS (
  SELECT "Pickup Plan ID", eventID
  FROM DOCEL_TruckPickupplanID
  WHERE objectID = (SELECT TruckID FROM IdentifiedTruck)
),
AssignedPlans AS (
SELECT e.eventID, Timestamp, tp."Pickup Plan ID", ROW_NUMBER() OVER (ORDER BY e.Timestamp) AS rn
FROM DOCEL_Events e
JOIN TruckPickupPlans tp on e.eventID = tp.eventID
ORDER BY Timestamp
)
SELECT "Pickup Plan ID"
FROM AssignedPlans
WHERE rn = 2;
"""
repeat = 1000

conn = sqlite3.connect(db_path)
cursor = conn.cursor()


durations = []

for i in range(repeat):
    start_time = time.time()
    cursor.execute(query)
    cursor.fetchall()  
    end_time = time.time()
    
    duration = (end_time - start_time) * 1000  
    durations.append(duration)
    # print(f"Run {i+1}: {duration:.4f} ms")


avg = statistics.mean(durations)
std_dev = statistics.stdev(durations)

print("\n=== Execution Statistics (DOCEL) ===")
print(f"Average execution time: {avg:.4f} ms")
print(f"Standard deviation: {std_dev:.4f} ms")


cursor.close()
conn.close()


=== Execution Statistics (DOCEL) ===
Average execution time: 0.6647 ms
Standard deviation: 0.0385 ms
